<div class="alert alert-secondary" role="alert">
    <h1 align = 'center'>San Francisco Crime Classification</h1>
    <h4 align = 'center' class="mb-0">Predict the category of crimes that occurred in the city by the bay</h4>
</div>

# 1: Importing The Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# 2: Load Dateset

In [ ]:
df = pd.read_csv('../input/sf-crime/train.csv.zip')
unseen_data = pd.read_csv('../input/sf-crime/test.csv.zip')
df.head()

# 3: EDA & Storytelling

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#check for duplicated values
df.duplicated().sum()

In [ ]:
#drop duplicated values
df.drop_duplicates(keep = 'first', inplace = True)
df.shape

In [ ]:
df.columns

In [ ]:
df.nunique()

In [ ]:
df.Category.value_counts()

In [ ]:
df.Resolution.value_counts()

In [ ]:
df['Dates'] = pd.to_datetime(df['Dates'])

df['year'] = df['Dates'].dt.year
df['month'] = df['Dates'].dt.month
df['day_of_month'] = df['Dates'].dt.day
df['day_of_week'] = df['Dates'].dt.day_of_week
df['hour'] = df['Dates'].dt.hour
df.drop(['Dates', 'DayOfWeek'], axis = 1, inplace = True)
df.dtypes

In [ ]:
df.columns = [item.lower() for item in df.columns]

In [ ]:
pd.crosstab(df['day_of_week'], df['category'])

In [ ]:
df.columns

In [ ]:
category_count = df.category.value_counts().to_frame().reset_index()
top5category = category_count.iloc[:5] 

df_cat = df.hour.value_counts().to_frame().sort_index()
plt.figure(figsize = (15, 5))
plt.plot(df_cat)

for item in top5category['index']:
    df_cat = df[df.category == item].hour.value_counts().to_frame().sort_index()
    plt.plot(df_cat)
plt.legend(['All Categories'] + list(top5category['index'].values))
plt.xlabel('hour')
_ = plt.ylabel('count')

In [ ]:
category_count = df.category.value_counts().to_frame().reset_index()
top5category = category_count.iloc[:5] 

df_cat = df.year.value_counts().to_frame().sort_index()
plt.figure(figsize = (15, 5))
plt.plot(df_cat)

for item in top5category['index']:
    df_cat = df[df.category == item].year.value_counts().to_frame().sort_index()
    plt.plot(df_cat)
plt.legend(['All Categories'] + list(top5category['index'].values))
plt.xlabel('year')
_ = plt.ylabel('count')

In [ ]:
df1 = df.groupby('pddistrict').count().reset_index()[['pddistrict', 'category']].sort_values('category', ascending = False)
plt.figure(figsize = (15, 5))
sns.barplot(x = 'pddistrict',y = 'category',data = df1, palette= 'Reds_r')

# 4: Data preprocessing

#### 4.1: Variable Encoding

In [ ]:
top200Des = df['descript'].value_counts()[:200].index
df['descript'] = df['descript'].apply(lambda x :x if x in top200Des else 'other' )
x = df.drop(['category', 'address'], axis = 1)
y = df['category']

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
x['descript'] = le.fit_transform(x['descript'])
x['pddistrict'] = le.fit_transform(x['pddistrict'])
x['resolution'] = le.fit_transform(x['resolution'])

#### 4.2: split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 1)
print('x train :', x_train.shape,'\t\tx test :', x_test.shape)
print('y train :', y_train.shape,'\t\ty test :', y_test.shape)

#### 4.3: feature scaling


In [ ]:
mms = MinMaxScaler()
x_train = mms.fit_transform(x_train)
x_test = mms.transform(x_test)

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# 5: Train your model


#### 5.1: KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)

#### 5.2: Random Forest

In [ ]:
rfc = RandomForestClassifier(random_state = 42)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)

#### 5.3: Decision Tree 

In [ ]:
dtc = DecisionTreeClassifier(random_state= 1)
dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)

#### 5.4: XGBoost 

In [ ]:
model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)